In [2]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
dir_path = "../Data/annotations/"
ano = "captions_train2017.json"
keyword_file = "kerwords.json"

In [3]:
with open(dir_path+ano, 'r') as open_ano:
    ano_json = json.load(open_ano)

with open(dir_path+ano, 'r') as open_key:
    keywords = json.load(open_key)
open_ano.close()
open_key.close()

In [4]:
ano_json['images'][0]

{'license': 3,
 'file_name': '000000391895.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000391895.jpg',
 'height': 360,
 'width': 640,
 'date_captured': '2013-11-14 11:18:45',
 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
 'id': 391895}

In [5]:
ano_json['annotations'][0]['caption']

'A bicycle replica with a clock as the front wheel.'

In [6]:
annotations = pd.DataFrame(ano_json['annotations'])

In [7]:
images = pd.DataFrame(ano_json['images'])

In [7]:
annotations#['caption'][0]

,image_id,id,caption
0,203564,37,A bicycle replica with a clock as the front wh...
1,322141,49,A room with blue walls and a white sink and door.
2,16977,89,A car that seems to be parked illegally behind...
3,106140,98,A large passenger airplane flying through the ...
4,106140,101,There is a GOL plane taking off in a partly cl...
...,...,...,...
591748,133071,829655,a slice of bread is covered with a sour cream ...
591749,410182,829658,A long plate hold some fries with some sliders...
591750,180285,829665,Two women sit and pose with stuffed animals.
591751,133071,829693,White Plate with a lot of guacamole and an ext...


In [8]:
keyword = ['fire',
           'forest fire',
           'firefighter',
           'burst',
           'explode',
           'explosion',
           'blast',
           'detonation',
           'detonated',
           'burned'
        ]
exclude_keyword = ['blazer','horse','candle','fireplace','fire place','ski','board','says','oven','wood','cook', 'old', 'living', 'plug', 'hyr', 'food', 'young','kitchen', 'extinguisher',
                  'fireworks','vintage','department', 'kid', 'boy', 'grill','fire station','fox','engine', 'hyd','fires']
k_image_id = []

In [440]:
keyword = '|'.join(keyword)

find = annotations[annotations['caption'].str.contains(keyword, case=False)]
print(f'final : {len(find)}')


exclude_keyword = '|'.join(exclude_keyword)
find = find[~find['caption'].str.contains(exclude_keyword, case=False)]
print(f'final : {len(find)}')
find.drop_duplicates(['image_id'], inplace=True ,ignore_index = True, keep = 'first')
print(f'droped duplicate images from multiple captions, After dropped: {len(find)}')
find.reset_index(inplace=True, drop=True)


final : 106
final : 91
droped duplicate images from multiple captions, After dropped: 67


##### 특정 키워드 caption에 포함된다면 hooking

In [13]:
find = annotations[annotations['caption'].str.contains('dododo', case=False)]

for i in keyword:
    find2 = annotations[annotations['caption'].str.contains(i, case=False)]
    find = pd.concat([find,find2], ignore_index=True)
    print(f'keyword : [{i}] // found: {len(find2)}')
print(f'final : {len(find)}')
## 결측지 유발 키워드 삭제
for i in exclude_keyword:
    origin_len = len(find)
    find = find[~find['caption'].str.contains(i, case=False)]
    print(f'excluded keyword:[{i}] found: {origin_len - len(find)}')
print(f'final : {len(find)}')
## 중복 이미지 삭제 (한 이미지에 대한 여러 caption)
find.drop_duplicates(['image_id'], inplace=True ,ignore_index = True, keep = 'first')
print(f'droped duplicate images from multiple captions, After dropped: {len(find)}')
## 인덱스 정렬
find.reset_index(inplace=True, drop=True)
#find

keyword : [fire] // found: 7298
keyword : [forest fire] // found: 3
keyword : [firefighter] // found: 43
keyword : [burst] // found: 11
keyword : [explode] // found: 3
keyword : [explosion] // found: 2
keyword : [blast] // found: 9
keyword : [detonation] // found: 0
keyword : [detonated] // found: 0
keyword : [burned] // found: 35
final : 7404
excluded keyword:[blazer] found: 0
excluded keyword:[horse] found: 20
excluded keyword:[candle] found: 16
excluded keyword:[fireplace] found: 866
excluded keyword:[fire place] found: 306
excluded keyword:[ski] found: 8
excluded keyword:[board] found: 30
excluded keyword:[says] found: 11
excluded keyword:[oven] found: 49
excluded keyword:[wood] found: 100
excluded keyword:[cook] found: 17
excluded keyword:[old] found: 300
excluded keyword:[living] found: 7
excluded keyword:[plug] found: 22
excluded keyword:[hyr] found: 21
excluded keyword:[food] found: 8
excluded keyword:[young] found: 74
excluded keyword:[kitchen] found: 13
excluded keyword:[exti

In [ ]:
## show img status
plt.figure(figsize=(16,500))
for i in range(len(find)):
    plt.subplot(100,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    k = str(find['image_id'][i]).zfill(12)
    path = "../Data/train2017/" + k +'.jpg'
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    plt.imshow(im_rgb, cmap=plt.cm.binary)
    plt.xlabel(find['caption'][i][:30] + '\n' + find['caption'][i][30:60] + '\n image_id : ' + str(find['image_id'][i]) , loc='left', fontsize=10)
plt.show()

In [417]:
tempest = find['image_id'].value_counts().index.sort_values()
tempest

Int64Index([  6733,  49551,  67788,  75308,  86467,  88029, 103276, 105367,
            118312, 127747, 135424, 142581, 144058, 157596, 160746, 162998,
            164491, 165160, 172727, 180857, 191546, 203062, 209102, 220072,
            234697, 237716, 248488, 253728, 261585, 271452, 275406, 280069,
            286899, 302713, 308610, 318533, 333363, 345590, 353400, 370417,
            372165, 376937, 381214, 400309, 406729, 429509, 435885, 436579,
            441045, 445298, 448759, 453533, 463222, 469410, 474964, 515422,
            517612, 525869, 538879, 540283, 541280, 555108, 558365, 563578,
            572889, 575348, 578655],
           dtype='int64')

In [418]:
len(tempest)

67

In [12]:
disaster_dataset = [581262, 382699, 374018, 184379, 140432, 564186, 351382, 216015, 483146, 454755, 39542, 180721,
                    391887, 369576, 393633, 244171, 520810, 368586, 375826, 423562, 436385, 51119, 278706, 350089,
                    731, 2453, 4489,   6845,  12647,  15112,  34525,  34985,  52308,  53719,  58624,  63147, 67868,
                    73187, 103404, 127006, 132791, 150161, 172255, 176440, 184606, 195992, 196099, 209063, 211263,
                    213826, 219472, 219657, 222245, 225246, 227248, 242762, 254418, 263453, 273001, 273174, 275027,
                    285605, 289846, 318415, 334936, 335217, 338703, 365170, 365295, 397617,398523, 416266, 422307, 442431,
                    450308, 456343,465601, 465638, 468602, 474862, 487299, 490978, 493828, 494404, 498679, 508672,
                    534194, 53719, 540943, 545924, 552573, 574456, 575753, 576700, 285679, 244674, 80462, 235221,
                    5169,  26097,  46118,  62589, 105640, 114701, 134981, 143490, 160163, 165061, 195951, 198463,
                    328569, 367838, 374241, 395765, 453954, 455044, 459003, 474653, 482320, 498241, 530876, 543332,
                    6733,  49551,  67788,  75308,  86467,  88029, 103276, 105367, 118312, 127747, 135424, 142581,
                    144058, 157596, 160746, 162998, 164491, 165160, 172727, 180857, 191546, 203062, 209102, 220072,
                    234697, 237716, 248488, 253728, 261585, 271452, 275406, 280069, 286899, 302713, 308610, 318533,
                    333363, 345590, 353400, 370417, 372165, 376937, 381214, 400309, 406729, 429509, 435885, 436579,
                    441045, 445298, 448759, 453533, 463222, 469410, 474964, 515422, 517612, 525869, 538879, 540283,
                    541280, 555108, 558365, 563578, 572889, 575348, 578655]
print(len(disaster_dataset))

189


In [ ]:
## show img status
plt.figure(figsize=(16,80))
for i in range(len(disaster_dataset)):
    plt.subplot(20,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    k = str(disaster_dataset[i]).zfill(12)
    path = "../Data/train2017/" + k +'.jpg'
    #print(path)
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    plt.imshow(im_rgb, cmap=plt.cm.binary)
    plt.xlabel('image_id : ' + str(disaster_dataset[i]) , loc='left', fontsize=10)
plt.show()

##### caption extraction for image generation

In [9]:
annotations['image_id'] = annotations['image_id'].apply(str)

In [10]:
annotations[annotations['image_id'].str.contains(str(581262), case=False)]['caption'].value_counts().index

Index(['A fire hydrant and some debris on the street. ',
       'The scene of a fire accident and a fire hydrant covered in ash. ',
       'this is the aftermath of a fire outside',
       'a old fire hydrogen that is next to a torn down buliding',
       'A fire hydrant beside a burned building in a city.'],
      dtype='object')

In [11]:
annotations[annotations['image_id'].str.contains(str(disaster_dataset[10]), case=False)]

,image_id,id,caption
311212,39542,62329,Firemen using the hose from the fire truck to ...
311229,39542,62731,Firefighters are using the hose on a firetruck...
311241,39542,62989,A fire truck spraying water into a bunch of tr...
311252,39542,63358,A fire truck putting out a fire next to a bridge.
311322,39542,64720,Three firemen shoot water from a tower cannon ...
322229,395425,340690,A semi trailer truck driving down the road wit...
322236,395425,340774,Large truck carrying three tractors on the str...
322238,395425,340813,Truck with three trackors on the back of it.
322312,395425,342049,Three large wheel trucks are on another truck.
322771,395425,351883,A large truck carries three tractors on a trai...


In [13]:
annotations[annotations['image_id']==str(disaster_dataset[10])]

,image_id,id,caption
311212,39542,62329,Firemen using the hose from the fire truck to ...
311229,39542,62731,Firefighters are using the hose on a firetruck...
311241,39542,62989,A fire truck spraying water into a bunch of tr...
311252,39542,63358,A fire truck putting out a fire next to a bridge.
311322,39542,64720,Three firemen shoot water from a tower cannon ...


In [1]:
keyword = ['fire',
           'forest fire',
           'firefighter',
           'burst',
           'explode',
           'explosion',
           'blast',
           'detonation',
           'detonated',
           'burned',
           
           'earthquake',
           'earth tremor',
           'vibration in the earth',
           'landslip',
           'avalanche',
           'landslide',
           'collapse',
           'cave in',
           'collapsed',
           'drought',
           
           'tempest',
           'windstorm',
           'rainstorm',
           'whirlwind',
           'flood',
           'inundate',
           
           'tsunami',
           'tidal wave',
           'hail',
           'hailstones',
           
           'hurricane',
           'typhoon',
           'cyclone',
           'tornado',
           'dust devil',
           'twister',
           
           'thunder',
           'lightning',
           'thunderstorm',
           'thunderbolt',
           
           'foggy day',
           'in the fog',
           'dense fog'
           
           'snow storm',
           'heavy snowfall',
           'heavy snow',
        ]
exclude_keyword = ['blazer','horse','candle','fireplace','fire place','ski','board','says','oven','wood','cook', 'old', 'living', 'plug', 'hyr', 'food', 'young','kitchen', 'extinguisher',
                  'fireworks','vintage','department', 'kid', 'boy', 'grill','fire station','fox','engine', 'hyd','fires']

In [14]:
captions_5 = annotations[annotations['image_id']== str(581262)]
captions_5

,image_id,id,caption
54317,581262,747205,A fire hydrant and some debris on the street.
54490,581262,752327,The scene of a fire accident and a fire hydran...
54692,581262,757819,this is the aftermath of a fire outside
54777,581262,760187,a old fire hydrogen that is next to a torn dow...
54839,581262,761597,A fire hydrant beside a burned building in a c...


In [19]:
captions = []

#captions_5 = annotations[annotations['image_id']== str(i)]['caption'].value_counts().index
for img_id in disaster_dataset:
    captions_5 = annotations[annotations['image_id']== str(img_id)]
    #print(captions_5)
    
    found = captions_5[captions_5['caption'].str.contains('dododo', case=False)]

    for i in keyword:
        found2 = captions_5[captions_5['caption'].str.contains(i, case=False)]
        found = pd.concat([found,found2], ignore_index=True)

    ## 결측지 유발 키워드 삭제
    for i in exclude_keyword:
        origin_len = len(found)
        found = found[~found['caption'].str.contains(i, case=False)]
    
    extracted = found['caption'].value_counts().index
    for j in extracted:
        captions.append(j.rstrip('\n').replace(",", ""))
        
print(f'num of captions : {len(captions)}')

num of captions : 302


#### delimiter ' ,'기준 문자열 안의 ,를 제거

In [20]:
# captions = [caption.replace(",", "") for caption in captions]
print(captions)

['this is the aftermath of a fire outside', 'A fire effect has been superimposed on a city street scene.', 'An airplane is dispensing fire repellent over a forest.', 'A plane is flying overhead disbursing fire retardant.', 'A jetliner sitting on top of an airport next to a fire.', 'A city street where a city bus and a car collided in an accident and fire and police personnel are around the vehicles.', 'Several fire fighters standing next to a car that a bus crashed into.', 'A firetruck spraying out water at a small airplane', 'Fire trucks spray water on a small plane on a busy airport runway.', 'Firefighters next to their trucks fight a blaze. ', 'a group of firefighters trying to stop a fire', 'A fire hose spurts water near a parked fire truck.', 'Hook and ladder fire truck at the scene of a night time fire.', 'a fireman a firetruck some hoses and a fire at night', 'A plane dropping fire retardant over a forest fire.', 'a plane drops fire retardant on a forest fire ', 'A bus waiting o